# Analytics Request Configuration

In [1]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account
from datetime import date
import pandas as pd 
from configs.settings import KEY_FILE_LOCATION, SCOPES, VIEW_ID

data_inicio = "2022-09-01"
data_fim = "2022-09-30"

def initialize_analyticsreporting():
    credentials = Credentials.from_service_account_file(KEY_FILE_LOCATION, scopes=SCOPES)
    analytics = build("analyticsreporting", "v4", credentials=credentials)
    return analytics

## Report Configurations

In [2]:
message_df = pd.DataFrame({ 
                            'eventAction': pd.Series(dtype='str'),
                            'totalEvents': pd.Series(dtype='int'),
                            'ViewID': pd.Series(dtype='int')
                          }) 

count_view = 0

while count_view < len(VIEW_ID):


    def get_report_visao_geral(analytics): # Definir Dimensions, Metrics e Segments
        return analytics.reports().batchGet(body={"reportRequests": [
        {
            "viewId": VIEW_ID[count_view],
            "dateRanges": [{"startDate": data_inicio, "endDate": data_fim}],
            "dimensions": [
                {"name": "ga:date"},
                {"name": "ga:eventAction"},
            ], 
            "metrics": [
                {"expression": "ga:totalEvents"},
            ],
        },

        ]}).execute()

    def response(response): # Transform Json to DF
        report = response.get("reports", [])[0] # expected just one report
        # headers
        header_dimensions = report.get("columnHeader", {}).get("dimensions", [])
        header_metrics = [value["name"] for value in report.get("columnHeader", {}).get("metricHeader", {}).get("metricHeaderEntries", [])]
        headers = header_dimensions + header_metrics
        headers = list(map((lambda x: x.split(":", 1)[-1]), headers)) # removes "ga:" from each column
        # values
        values = []
        rows = report.get("data", {}).get("rows", [])
        for row in rows:
            values_dimensions = row.get("dimensions", [])
            values_metrics = row.get("metrics", [])[0].get("values", [])
            values.append(values_dimensions + values_metrics)
        # to dataframe
        df = pd.DataFrame(columns=headers, data=values)
        return df

    analytics = initialize_analyticsreporting()
    response_visao_geral = get_report_visao_geral(analytics)
    df = response(response_visao_geral)

    df['ViewID'] = VIEW_ID[count_view]

    message_df = pd.concat([message_df, df], ignore_index=True)
    VIEW_ID
    count_view = count_view + 1

message_df = message_df.astype({'totalEvents':'int','eventAction':'string', 'ViewID':'string', 'date':'string'})
message_df = pd.pivot_table(data= message_df, index = ['date', 'ViewID'], columns='eventAction', values='totalEvents')

In [3]:
user_df = pd.DataFrame({ 
                        'ViewID': pd.Series(dtype='int'),
                       }) 

count_view = 0

while count_view < len(VIEW_ID):


    def get_report_visao_geral(analytics): # Definir Dimensions, Metrics e Segments
        return analytics.reports().batchGet(body={"reportRequests": [
        {
            "viewId": VIEW_ID[count_view],
            "dateRanges": [{"startDate": data_inicio, "endDate": data_fim}],
            "dimensions": [
                {"name": "ga:date"}
            ], 
            "metrics": [
                {"expression": "ga:users"},
                {"expression": "ga:sessions"},
            ],
        },

        ]}).execute()

    def response(response): # Transform Json to DF
        report = response.get("reports", [])[0] # expected just one report
        # headers
        header_dimensions = report.get("columnHeader", {}).get("dimensions", [])
        header_metrics = [value["name"] for value in report.get("columnHeader", {}).get("metricHeader", {}).get("metricHeaderEntries", [])]
        headers = header_dimensions + header_metrics
        headers = list(map((lambda x: x.split(":", 1)[-1]), headers)) # removes "ga:" from each column
        # values
        values = []
        rows = report.get("data", {}).get("rows", [])
        for row in rows:
            values_dimensions = row.get("dimensions", [])
            values_metrics = row.get("metrics", [])[0].get("values", [])
            values.append(values_dimensions + values_metrics)
        # to dataframe
        df = pd.DataFrame(columns=headers, data=values)
        return df

    analytics = initialize_analyticsreporting()
    response_visao_geral = get_report_visao_geral(analytics)
    df = response(response_visao_geral)

    df['ViewID'] = VIEW_ID[count_view]

    user_df = pd.concat([user_df, df], ignore_index=True)
    
    count_view = count_view + 1

user_df = user_df.astype({'users':'int','sessions':'int', 'ViewID':'string', 'date':'string'})

In [4]:
final_df = message_df.merge(user_df,how='right',left_on=['ViewID', 'date'],right_on=['ViewID', 'date'])